In [ ]:
reload_lamb()

# Lexical ambiguity

Most natural language content words are ambiguous in an "accidental" way -- that is, there is no rhyme or reason why certain meanings coincide in vertain forms.  The classic example of this is the word "bank", which has many unrelated senses, including the two put into `bank1` and `bank2` below.  This method just separates ambiguous meanings.

In [ ]:
%%lamb reset
||bank1|| = L x_e : Riverbank(x)
||bank2|| = L x_e : Moneybank(x)
||the|| = L f_<e,t> : Iota x_e : f(x)

The lambda notebook can directly handle ambiguity in lexical items.  The simplest way is to use a cell magic option, `ambiguity` -- when this is active, adding more definitions to the same lexical item will produce ambiguity.  When `ambiguity` is not active (it is off by default), new definitions will replace old definitions.  An ambiguous lexical item can be used anywhere a regular item can, and will produce multiple derivations (like an ambiguous composition result).

Some care should be taken with this, and you typically want to combine it with the `reset` option, so that repeatedly running the cell doesn't keep tacking on more entries.  (There is no check for equality, so you can easily end up with 20 versions of the same entry.)

In [ ]:
%%lamb reset,ambiguity
||bank|| = L x_e : Riverbank(x)
||bank|| = L x_e : Moneybank(x)
||the|| = L f_<e,t> : Iota x_e : f(x)

In [ ]:
(the * bank).tree()

A second way of dealing with ambiguous items is by explicit indexing.  A sense index can be supplied in brackets after the item name; the behavior of this index is the same as for python indices with one exception.  To add a new sense, you can use `[*]`.

Note that one way in which it behaves the same as in python is that indices must correspond to existing entries, and are not treated as sparse.  So you must use `[*]` to append, and other indices can only edit in place.

In [ ]:
%%lamb
||test|| = L x_e : Exam(x) # this line will reset the entry for ||test|| each time the cell is run
||test[*]|| = L x_e : CricketMatch(x)
||test[*]|| = L y_e : L x_e : GaveExam(x,y)

There are no type constraints on ambiguity, so multiple (even incompatible) types is perfectly fine.  Composition will typically eliminate cases of type mismatches (just watch for surprises!).  For example:

In [ ]:
the * test

Ambiguous items can be indexed directly as python objects to get regular `Item`s, so if you want to compose particular senses together you can do this with indices.  Slicing is implemented for this case as well.  (**N.b.** slicing is not implemented for composition results in general, nor for assignment in `%%lamb` cells.  For the former it will produce a list, for the latter a parse error.)

In [ ]:
the * test[1]

In [ ]:
test[0:2]

### Ambiguity via ad-hoc polymorphism

There is a second way that ambiguity can be implemented, using the mechanism for ad-hoc polymorphism (*disjunctive types*).  This can only be used if the ambiguity involves different types, as ad-hoc polymorphism requires single fixed expressions at any particular type.  The following entry uses ad-hoc polymorphism to write an entry for `a` that works as both a determiner and in predicative sentences.  (The standard semantics 1 approach is to take this to simply be an ambiguity.)

In [ ]:
te("Disjunctive((L f_<e,t> : f), (L f_<e,t> : L g_<e,t> : Exists x_e : f(x) & g(x)))")

In [ ]:
%%lamb
||a|| = Disjunctive((L f_<e,t> : f), (L f_<e,t> : L g_<e,t> : Exists x_e : f(x) & g(x)))
||cat|| = L x_e : Cat(x)
||alfonso|| = A_e
||danced|| = L x_e : Dance(x)

In [ ]:
a * cat

The following uses the quantificational version of `a`.  It actually doesn't force the quantificational version, because the predicative version can still compose with `danced` via PM -- this could be filtered out by assuming sentences most be type `t`.  (There is a lesson about the pitfalls of ambiguity here.)

In [ ]:
(a * cat) * danced

The following forces the predicative interpretation of `a`.

In [ ]:
alfonso * (a * cat)